# MySQL
데이터를 데이터베이스에 저장한 뒤에, SQL 쿼리를 활용해 출력을 해보겠습니다.
____

In [3]:
!pip install mysql-connector

     |████████████████████████████████| 11.9 MB 14.6 MB/s eta 0:00:01
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp38-cp38-linux_x86_64.whl size=247947 sha256=c831d075fe9723839a8e6b8776c85774af4eb9c19168ef31bcb7abe3817dfaea
  Stored in directory: /opt/ml/.cache/pip/wheels/57/e4/98/5feafb5c393dd2540e44b064a6f95832990d543e5b4f53ea8f
Successfully built mysql-connector


## 1. 데이터 불러오기

In [1]:
import pandas as pd
df = pd.read_csv("movie_doc.csv",sep="\t")
df.head()

,id,title,story,gender,link,image
0,0,인비저블 게스트,"의문의 습격으로 살해 당한 ‘로라’ ‘아드리안’은 연인의 죽음에 절망하고, 범인은 ...",N,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20170828_179/1...
1,1,"나, 다니엘 블레이크",평생을 성실하게 목수로 살아가던 다니엘은 지병인 심장병이 악화되어 일을 계속 해나갈...,F,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20161117_246/1...
2,2,국가부도의 날,"1997년, 대한민국 최고의 경제 호황을 믿어 의심치 않았던 그때, 곧 엄청난 경제...",M,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20181105_232/1...
3,3,당갈,전직 레슬링 선수였던 ‘마하비르 싱 포갓(아미르 칸)’은 아버지의 반대로 금메달의 ...,F,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20180329_2/152...
4,4,스파이더맨: 파 프롬 홈,"‘엔드게임’ 이후 변화된 세상, 스파이더맨 ‘피터 파커’는 학교 친구들과 유럽 여행...",M,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20190527_181/1...


## 2. MySQL을 활용한 데이터 저장

In [2]:
import mysql.connector

### 2-1. Database 생성

In [4]:

HOST = "localhost"
USER = "root"
PASSWORD = "Boost111@"
mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD
)

mycursor = mydb.cursor()

# mycursor.execute("DROP DATABASE movie_db")
try:
    mycursor.execute("CREATE DATABASE movie_db default character set utf8 collate utf8_general_ci;")
except:
    pass

### 2-2. Table 생성

In [5]:
mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD,
  database="movie_db"
)

mycursor = mydb.cursor()

In [6]:

try:
    mycursor.execute("DROP TABLE movie;")
except:
    pass
mycursor.execute("""
                    CREATE TABLE movie 
                    (id VARCHAR(255) UNIQUE, story TEXT, title VARCHAR(255), link VARCHAR(255), image VARCHAR(255))"""
                )


In [7]:
for row_dict in df.to_dict(orient="records"):
    print(row_dict)
    break

{'id': 0, 'title': '인비저블 게스트', 'story': '의문의 습격으로 살해 당한 ‘로라’ ‘아드리안’은 연인의 죽음에 절망하고, 범인은 흔적도 없이 사라졌다. 유력한 용의자로 누명을 쓴 ‘아드리안’은 승률 100%의 변호사 ‘버지니아’를 선임한다. 그리고 자신의 무죄를 입증하기 위해 고군분투하던 중 과거 그와 ‘로라’가 은폐한 교통사고와 숨겨진 연관성을 찾게 되는데… 남은 시간은 단 3시간, 사건을 재구성해 무죄를 입증해야 한다!', 'gender': 'N', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=159516', 'image': 'https://movie-phinf.pstatic.net/20170828_179/1503887362732snsIK_JPEG/movie_image.jpg?type=f67'}


### 2-3. Row 삽입

In [8]:
for row_dict in df.to_dict(orient="records"):
#     print(row_dict)
    movie_id = str(row_dict['id'])
    title = row_dict['title']
    story = row_dict['story']
    link = row_dict['link']
    image = row_dict['image']

    sql = "INSERT INTO movie (id, story, title, link, image) VALUES (%s, %s, %s, %s, %s)"
    val = (movie_id, story, title, link, image)
    mycursor.execute(sql, val)

mydb.commit()



### 2-4. 삽입된 Row 출력

In [9]:
sql = "SELECT * FROM movie"
mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)
    break

{'id': '0', 'story': '의문의 습격으로 살해 당한 ‘로라’ ‘아드리안’은 연인의 죽음에 절망하고, 범인은 흔적도 없이 사라졌다. 유력한 용의자로 누명을 쓴 ‘아드리안’은 승률 100%의 변호사 ‘버지니아’를 선임한다. 그리고 자신의 무죄를 입증하기 위해 고군분투하던 중 과거 그와 ‘로라’가 은폐한 교통사고와 숨겨진 연관성을 찾게 되는데… 남은 시간은 단 3시간, 사건을 재구성해 무죄를 입증해야 한다!', 'title': '인비저블 게스트', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=159516', 'image': 'https://movie-phinf.pstatic.net/20170828_179/1503887362732snsIK_JPEG/movie_image.jpg?type=f67'}


In [10]:

keyword = "암살자"
sql = "SELECT * FROM movie WHERE story LIKE '%{}%'".format(keyword)

mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)

{'id': '41', 'story': "고도의 훈련을 받은 최고의 암살요원 제이슨 본. 사고로 잃었던 기억을 단편적으로 되살리던 제이슨 본은 자신을 암살자로 만든 이들을 찾던 중 ‘블랙브라이어’라는 존재를 알게 된다. ‘블랙브라이어’는 비밀요원을 양성해내던 '트레드스톤'이 국방부 산하의 극비조직으로 재편되면서 더욱 막강한 파워를 가지게 된 비밀기관. 그들에게 자신들의 비밀병기 1호이자 진실을 알고 있는 유일한 인물인 제이슨 본은 반드시 제거해야 하는 대상이다. 니키의 도움으로 블랙브라이어의 실체를 알게 된 제이슨 본은 런던, 마드리드, 모로코 그리고 뉴욕까지 전세계를 실시간 통제하며 자신을 제거하고 비밀을 은폐하려는 조직과 숨막히는 대결을 시작하는데…", 'title': '본 얼티메이텀', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=59075', 'image': 'https://movie-phinf.pstatic.net/20160628_2/1467080136011unJpU_JPEG/movie_image.jpg?type=f67'}
{'id': '67', 'story': '업계 최고의 레전드 킬러 ‘존 윅’은 과거를 뒤로한 채 은퇴를 선언하지만, 과거 자신의 목숨을 구했던 옛 동료와 피로 맺은 암살자들의 룰에 의해 로마로 향한다. ‘국제 암살자 연합’을 탈취하려는 옛 동료의 계획으로 ‘존 윅’은 함정에 빠지게 되고, 전세계 암살자들의 총구는 그를 향하는데...', 'title': '존 윅 - 리로드', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=143932', 'image': 'https://movie-phinf.pstatic.net/20200807_193/1596789703698oLgpX_JPEG/movie_image.jpg?type=f67'}


### Q.여러키워드를 포함시키려면?

In [12]:


keyword = "뉴욕 암살"
sql = "SELECT * FROM movie WHERE "
operator = "AND"
for word in keyword.split():
    sql += "story LIKE '%{}%' {} ".format(word, operator)

sql = sql[:sql.rfind(operator)]+";"
print(sql)

mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)

SELECT * FROM movie WHERE story LIKE '%뉴욕%' AND story LIKE '%암살%' ;
{'id': '41', 'story': "고도의 훈련을 받은 최고의 암살요원 제이슨 본. 사고로 잃었던 기억을 단편적으로 되살리던 제이슨 본은 자신을 암살자로 만든 이들을 찾던 중 ‘블랙브라이어’라는 존재를 알게 된다. ‘블랙브라이어’는 비밀요원을 양성해내던 '트레드스톤'이 국방부 산하의 극비조직으로 재편되면서 더욱 막강한 파워를 가지게 된 비밀기관. 그들에게 자신들의 비밀병기 1호이자 진실을 알고 있는 유일한 인물인 제이슨 본은 반드시 제거해야 하는 대상이다. 니키의 도움으로 블랙브라이어의 실체를 알게 된 제이슨 본은 런던, 마드리드, 모로코 그리고 뉴욕까지 전세계를 실시간 통제하며 자신을 제거하고 비밀을 은폐하려는 조직과 숨막히는 대결을 시작하는데…", 'title': '본 얼티메이텀', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=59075', 'image': 'https://movie-phinf.pstatic.net/20160628_2/1467080136011unJpU_JPEG/movie_image.jpg?type=f67'}


## 5. Connection 끊어주기

In [13]:
mycursor.close()
mydb.close()